# Introduction
In this Notebook, we go through instantiating a backend server within a notebook, which allows more in-depth and realtime modifications to the agents and the network while the flask server is running.

# Import and Load Environment Variables

In [1]:
from dotenv import load_dotenv
import asyncio
from threading import Thread
import sys
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field

path_to_root = Path.cwd().parent
sys.path.append(str(path_to_root))

from agents.InboxAgent import InboxAgent
from agents.UserProxyAgent import UserProxyAgent
from network.AgentNetwork import AgentNetwork
from server.NetworkServer import NetworkServer


# load environment variables from .env
load_dotenv()

True

# Instantiate the AgentNetwork and Start the Server

The AgentNetwork is what manages all AI Agents and UserProxyAgents. It also manages the creation and connections between agents. 'available_agent_instantiations' is a list of possible agents the frontend can instantiate. Here we place the only two currently existing agent bases which are the default InboxAgent and the UserProxyAgent.

The NetworkServer is what manages the communication with the frontend GUI [agent-network-visualizer](https://github.com/awkyu/AsyncAgents/tree/main/agent-network-visualizer). We create a Flask server to establish communication and run it in a thread to allow realtime updates later in this notebook. We get the event loop so that other agents may access the loop to run asynchronously. 'allow_unsafe_werkzeug' is set to True to allow us to run it in the notebook, but should not be used in a production setting.

In [2]:
agent_network = AgentNetwork(available_agent_instantiations=[InboxAgent, UserProxyAgent])

In [3]:
loop = asyncio.get_event_loop()
network_server = NetworkServer(agent_network, event_loop=loop)
flask_thread = Thread(target=network_server.run, kwargs={'use_reloader': False, 'allow_unsafe_werkzeug': True})
flask_thread.start()

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app 'server.AgentNetworkSocket'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


# Instantiating Custom Agents in Realtime

### Define the language model to use and agent tools

Step 1: Initialize the language model; currently only GPT-4 works, but with better prompt engineering, GPT-3 should work too\
Step 2: Define the tools (functions) that the assistant can use\
Step 3: We use the StructuredTool class to define the tools, which allows us to define a schema for the arguments (which allows the model to understand the tool better)\
Step 4: create a list of tools

In [4]:
# Step 1: Initialize the language model; currently only GPT-4 works, but with better prompt engineering, GPT-3 should work too
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

# Step 2: Define the tools (functions) that the assistant can use
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# Step 3: We use the StructuredTool class to define the tools, which allows us to define a schema for the arguments (which allows the model to understand the tool better)
class GetWordLength(BaseModel):
    word: str = Field(
        description="The word you want to get the length of"
    )

# Step 4: create a list of tools
tools = [
    StructuredTool.from_function(
        get_word_length,
        name="get_word_length",
        description="Returns the length of a word.",
        args_schema=GetWordLength,
    ),
]

127.0.0.1 - - [12/Jan/2024 03:39:04] "GET /socket.io/?EIO=4&transport=polling&t=OpyqnZd HTTP/1.1" 200 -


127.0.0.1 - - [12/Jan/2024 03:39:04] "GET /socket.io/?EIO=4&transport=polling&t=Opyqnbg HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:04] "POST /socket.io/?EIO=4&transport=polling&t=Opyqo2N&sid=30GK-pE5CePMtOnGAAAA HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:04] "GET /socket.io/?EIO=4&transport=polling&t=Opyqo2P&sid=30GK-pE5CePMtOnGAAAA HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:04] "POST /socket.io/?EIO=4&transport=polling&t=Opyqo4c&sid=jtLDDkLLsn23BOppAAAB HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:04] "GET /socket.io/?EIO=4&transport=polling&t=Opyqo4d&sid=jtLDDkLLsn23BOppAAAB HTTP/1.1" 200 -


Client connected
{'nodes': [], 'links': []}
Client connected
{'nodes': [], 'links': []}


### Instantiate Agents and UserProxy

We instantiate 5 InboxAgents.\
llm is the defined language model above. If None, then it will default to a GPT-4 model ("gpt-4-1106-preview")\
name/type dictate their secondary color in the frontend GUI. 'tools' is the list of tools you want to add (put None or [] if no tools).\
description is the description of the AI agent assistant\
oai is used for OpenAI models (which is the only one supported for now).\
response_counter is how many messages the agent will process before needing to be reactivated
idle_counter is how many seconds the agent will sit idle (no messages in its inbox) before deactivating
\
\
We also instantiate two UserProxyAgents for communication.

In [5]:
agent1 = InboxAgent(
    name="jokebot_teller",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to tell jokes with other AI agents by sending messages.",
    oai=True,
    response_counter=3,
    idle_counter=60,
)

agent2 = InboxAgent(
    name="jokebot_responder",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to laugh at jokes with other AI agents. you like to send jokes back via messages too and like to pass them on to others that may not have not heard them!",
    oai=True,
    response_counter=3,
    idle_counter=60,
)

agent3 = InboxAgent(
    name="jokebot_responder",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to laugh at jokes with other AI agents. you like to send jokes back via messages too and like to pass them on to others that may not have not heard them!",
    oai=True,
    response_counter=3,
    idle_counter=60,
)

agent4 = InboxAgent(
    name="jokebot_responder",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to laugh at jokes with other AI agents. you like to send jokes back via messages too and like to pass them on to others that may not have not heard them!",
    oai=True,
    response_counter=3,
    idle_counter=60,
)

agent5 = InboxAgent(
    name="jokebot_responder",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to laugh at jokes with other AI agents. you like to send jokes back via messages too and like to pass them on to others that may not have not heard them!",
    oai=True,
    response_counter=3,
    idle_counter=60,
)

user1 = UserProxyAgent()
user2 = UserProxyAgent()

### Add to Agent Network

We add all 7 agents (5 InboxAgents and 2 UserProxyAgents) to the Agent Network and then connect them in the desired graph network.

In [ ]:
# Add agents to the network
agent_network.add_agent(agent1)
agent_network.add_agent(agent2)
agent_network.add_agent(agent3)
agent_network.add_agent(agent4)
agent_network.add_agent(agent5)
agent_network.add_agent(user1)
agent_network.add_agent(user2)

# Connect agents to each other
agent_network.connect_agents(agent1, agent2)
agent_network.connect_agents(user1, agent1)
agent_network.connect_agents(agent1, agent3)
agent_network.connect_agents(agent2, agent4)
agent_network.connect_agents(agent3, agent5)
agent_network.connect_agents(agent4, agent5)
agent_network.connect_agents(agent5, user2)

127.0.0.1 - - [12/Jan/2024 03:39:09] "GET /socket.io/?EIO=4&transport=websocket&sid=jtLDDkLLsn23BOppAAAB HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [12/Jan/2024 03:39:10] "GET /socket.io/?EIO=4&transport=polling&t=Opyqpk1 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:11] "GET /socket.io/?EIO=4&transport=polling&t=OpyqpkA HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:11] "POST /socket.io/?EIO=4&transport=polling&t=OpyqppC&sid=HTrZ_35vodyT7GzZAAAF HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:11] "GET /socket.io/?EIO=4&transport=polling&t=OpyqppE&sid=HTrZ_35vodyT7GzZAAAF HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:39:11] "GET /socket.io/?EIO=4&transport=polling&t=Opyqpum&sid=HTrZ_35vodyT7GzZAAAF HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'label': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'type': 'jokebot_teller', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'label': 'jokebot_responder_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d', 'label': 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'User_38771e67-e7da-403b-af20-9ab2efedc084', 'label': 'User_38771e67-e7da-403b-af20-9ab2efedc084', 'type': 'User', 'running': True, 'status': 'Running'}], 'links': [{'id': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995_jokebot_responder_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'source': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'target': 'jokebot_respo

127.0.0.1 - - [12/Jan/2024 03:39:12] "GET /socket.io/?EIO=4&transport=polling&t=Opyqpz0&sid=HTrZ_35vodyT7GzZAAAF HTTP/1.1" 200 -


...

Client connected
{'nodes': [{'id': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'label': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'type': 'jokebot_teller', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d', 'label': 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'User_38771e67-e7da-403b-af20-9ab2efedc084', 'label': 'User_38771e67-e7da-403b-af20-9ab2efedc084', 'type': 'User', 'running': True, 'status': 'Running'}, {'id': 'jokebot_responder_2_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'label': 'jokebot_responder_2_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'type': 'jokebot_responder_2', 'running': False, 'status': 'Stopped'}, {'id': 'User_8255891c-fc6c-4c68-9266-ec4f7a578976', 'label': 'User_8255891c-fc6c-4c68-9266-ec4f7a578976', 'type': 'User', 'running': True, 'status': 'Running'}, {'id': 'test_dcc8f798-3dd2-44ac-81d6-633aa180d28d'

127.0.0.1 - - [12/Jan/2024 03:51:36] "GET /socket.io/?EIO=4&transport=websocket&sid=HTrZ_35vodyT7GzZAAAF HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:51:36] "GET /socket.io/?EIO=4&transport=polling&t=Opytfh5 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:51:36] "GET /socket.io/?EIO=4&transport=polling&t=OpytfhI&sid=WcwwlQ2sj-n8S0wuAAAK HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:51:36] "POST /socket.io/?EIO=4&transport=polling&t=Opytfkg&sid=Wrrig7iU7xqblwLvAAAM HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 03:51:36] "GET /socket.io/?EIO=4&transport=polling&t=Opytfkh&sid=Wrrig7iU7xqblwLvAAAM HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'label': 'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995', 'type': 'jokebot_teller', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d', 'label': 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'User_38771e67-e7da-403b-af20-9ab2efedc084', 'label': 'User_38771e67-e7da-403b-af20-9ab2efedc084', 'type': 'User', 'running': True, 'status': 'Running'}, {'id': 'jokebot_responder_2_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'label': 'jokebot_responder_2_49d9d393-58b6-4edc-9143-2ba97a1d77c2', 'type': 'jokebot_responder_2', 'running': False, 'status': 'Stopped'}, {'id': 'User_8255891c-fc6c-4c68-9266-ec4f7a578976', 'label': 'User_8255891c-fc6c-4c68-9266-ec4f7a578976', 'type': 'User', 'running': True, 'status': 'Running'}, {'id': 'test_dcc8f798-3dd2-44ac-81d6-633aa180d28d'

127.0.0.1 - - [12/Jan/2024 03:51:37] "GET /socket.io/?EIO=4&transport=polling&t=Opytfqx&sid=Wrrig7iU7xqblwLvAAAM HTTP/1.1" 200 -


{'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995': <agents.InboxAgent.InboxAgent object at 0x000001B468C36CD0>, 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d': <agents.InboxAgent.InboxAgent object at 0x000001B468C36A90>, 'User_38771e67-e7da-403b-af20-9ab2efedc084': <agents.UserProxyAgent.UserProxyAgent object at 0x000001B468C36B80>, 'jokebot_responder_2_49d9d393-58b6-4edc-9143-2ba97a1d77c2': <agents.InboxAgent.InboxAgent object at 0x000001B468C36C70>, 'User_8255891c-fc6c-4c68-9266-ec4f7a578976': <agents.UserProxyAgent.UserProxyAgent object at 0x000001B468CC09D0>, 'test_dcc8f798-3dd2-44ac-81d6-633aa180d28d': <agents.InboxAgent.InboxAgent object at 0x000001B468CA44F0>}
{'jokebot_teller_888044cc-1ef5-40ff-ac1e-b139bdd52995': <agents.InboxAgent.InboxAgent object at 0x000001B468C36CD0>, 'jokebot_responder_2642e700-eca2-4491-a1d9-d0b842d5958d': <agents.InboxAgent.InboxAgent object at 0x000001B468C36A90>, 'User_38771e67-e7da-403b-af20-9ab2efedc084': <agents.UserProxyAgent.UserP

### Start/Stop the agents to run them asynchronously and send messages within the notebook

This is also possible in the frontend GUI

In [ ]:
agent1.start()
agent2.start()
agent3.start()

{'jokebot_teller_3e02a8c7-1c66-4053-9ddd-c5ee1404c459': <InboxAgent.InboxAgent object at 0x0000021BE317CCD0>, 'jokebot_responder_ea55748a-7dde-4e40-968d-4da5a140d44d': <InboxAgent.InboxAgent object at 0x0000021BE317CB50>, 'jokebot_responder_b0335572-94f5-4aef-be02-b823493c1c23': <InboxAgent.InboxAgent object at 0x0000021BE317CA30>, 'User_bd09e4e9-5549-40a4-b07a-2973058cea94': <UserProxyAgent.UserProxyAgent object at 0x0000021BE317CC70>}


127.0.0.1 - - [12/Jan/2024 02:31:56] "GET /socket.io/?EIO=4&transport=websocket&sid=cF6u_Im8q3MRhzLqAAAC HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [12/Jan/2024 02:31:58] "GET /socket.io/?EIO=4&transport=polling&t=OpybR6g HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 02:31:58] "GET /socket.io/?EIO=4&transport=polling&t=OpybR6n HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 02:31:58] "POST /socket.io/?EIO=4&transport=polling&t=OpybRBx&sid=ftX8_A6VX4VeA5dBAAAF HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2024 02:31:58] "GET /socket.io/?EIO=4&transport=polling&t=OpybRBy&sid=ftX8_A6VX4VeA5dBAAAF HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_3e02a8c7-1c66-4053-9ddd-c5ee1404c459', 'label': 'jokebot_teller_3e02a8c7-1c66-4053-9ddd-c5ee1404c459', 'type': 'jokebot_teller', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_ea55748a-7dde-4e40-968d-4da5a140d44d', 'label': 'jokebot_responder_ea55748a-7dde-4e40-968d-4da5a140d44d', 'type': 'jokebot_responder', 'running': True, 'status': 'Idle'}, {'id': 'jokebot_responder_b0335572-94f5-4aef-be02-b823493c1c23', 'label': 'jokebot_responder_b0335572-94f5-4aef-be02-b823493c1c23', 'type': 'jokebot_responder', 'running': True, 'status': 'Idle'}, {'id': 'User_bd09e4e9-5549-40a4-b07a-2973058cea94', 'label': 'User_bd09e4e9-5549-40a4-b07a-2973058cea94', 'type': 'User', 'running': True, 'status': 'Running'}], 'links': [{'id': 'jokebot_teller_3e02a8c7-1c66-4053-9ddd-c5ee1404c459_jokebot_responder_ea55748a-7dde-4e40-968d-4da5a140d44d', 'source': 'jokebot_teller_3e02a8c7-1c66-4053-9ddd-c5ee1404c459', 'target': 'jokebot_responder_ea5

In [ ]:
user1.send_message("tell me a joke", agent1.get_description_json()['name_uuid'])

In [ ]:
agent1.stop()
agent2.stop()
agent3.stop()

127.0.0.1 - - [09/Jan/2024 15:49:38] "GET /socket.io/?EIO=4&transport=websocket&sid=I5NoeTI7qslkDtm2AAAD HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [09/Jan/2024 15:49:39] "GET /socket.io/?EIO=4&transport=polling&t=Opl_Eby HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:49:39] "POST /socket.io/?EIO=4&transport=polling&t=Opl_Eh2&sid=qLi9Zr06UouvBposAAAF HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:49:39] "GET /socket.io/?EIO=4&transport=polling&t=Opl_Eh3&sid=qLi9Zr06UouvBposAAAF HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'label': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'label': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'label': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'label': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360_jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'source': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'target': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'curvature': 0.2, 'rotation': 3.141592653589

127.0.0.1 - - [09/Jan/2024 15:50:06] "GET /socket.io/?EIO=4&transport=websocket&sid=qLi9Zr06UouvBposAAAF HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:50:06] "GET /socket.io/?EIO=4&transport=polling&t=Opl_LW3 HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [09/Jan/2024 15:50:07] "POST /socket.io/?EIO=4&transport=polling&t=Opl_LWV&sid=MaVVubId23w0bpzqAAAH HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:50:07] "GET /socket.io/?EIO=4&transport=polling&t=Opl_LWY&sid=MaVVubId23w0bpzqAAAH HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'label': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'label': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'label': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'label': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360_jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'source': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'target': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'curvature': 0.2, 'rotation': 3.141592653589